**Read the libraries**

In [80]:
import os
import psycopg2 #PosgreSQL adapter for Python
from dotenv import load_dotenv 
import pandas as pd
from io import StringIO

**Create a dataframe**

In [81]:
#create a dataframe
data = {
    'date': ['2023-01-01', '2023-01-02', '2023-01-03'],
    'temperature': [25.0, 26.5, 24.8],
}

df = pd.DataFrame(data)
df

,date,temperature
0,2023-01-01,25.0
1,2023-01-02,26.5
2,2023-01-03,24.8


**Create a connection with the database**

In [82]:
#loads .env file = connects with the database
load_dotenv()

#set up connection with the database (psycopg2 is faster than Alchemy)
url=os.getenv("DATABASE_URL")
connection=psycopg2.connect(url)

**Create a table**

In [83]:
#Open a cursor to perform database operations
cursor = connection.cursor()

#define the table
table_name='temperature_table'

#Create a table
CREATE_FIRST_TABLE=f"""CREATE TABLE IF NOT EXISTS {table_name}(
            id SERIAL PRIMARY KEY,
            date DATE,
            temperature DECIMAL
            )"""

#Excute the operation
cursor.execute(CREATE_FIRST_TABLE)

#Commit changes to the database
connection.commit()

**Delete previous data from the table**

In [84]:
#delete previous data from the database

DELETE_DATA = f"DELETE FROM {table_name}"

#Execute the DELETE query
cursor.execute(DELETE_DATA)

#commit
connection.commit()

**Insert data into the table**

In [85]:
#Insert a data into a table
# Create a CSV string from the DataFrame (excluding the id column)
csv_data = df[['date', 'temperature']].to_csv(index=False, header=False)

# Use StringIO to create a file-like object from the CSV string
csv_file = StringIO(csv_data)

# Use the COPY command to load data from the CSV file into the table
cursor.copy_from(csv_file, 'temperature_table', sep=',', columns=('date', 'temperature'))

#commit
connection.commit()

**Read the data from the table and insert them into a dataframe**

In [86]:
#Read data from the table
READ_DATA=f"SELECT* FROM  {table_name}"


#Execute the query
cursor.execute(READ_DATA)

# Fetch all rows at once
rows = cursor.fetchall()


#Insert data into a dataframe
columns = [desc[0] for desc in cursor.description] #get names of the columns
df = pd.DataFrame(rows, columns=columns)
df

,id,date,temperature
0,28,2023-01-01,25.0
1,29,2023-01-02,26.5
2,30,2023-01-03,24.8


**Close the connection with the database**

In [87]:
# Close the cursor and connection 
if cursor:
    cursor.close()

if connection:
    connection.close()